# Задание 3.4

## Курузов Илья, 676

Извлекаем данные.

In [1]:
path = "data/mt"

In [2]:
import os

In [3]:
classes = ["_1.txt", "_2.txt", "_gold.txt"]
texts = {i:{} for i in classes}
for i in os.listdir(path):
    for j in classes:
        if j in i:
            with open(path+"/"+i, "r") as f:
                # Переводим в нижний регистр
                texts[j][i[:i.index(j)]] = f.read().lower()
                f.close()

In [4]:
print("Количество переведенных текстов", len(texts[j]))

Количество переведенных текстов 5


Разобъем тексты по предложениям

In [5]:
keys = list(texts[classes[0]].keys())
keys

['en_ru', 'ru_kk', 'en_kk', 'ru_de', 'de_en']

In [6]:
new_texts = {i:{} for i in classes}
for i in classes:
    cur_texts = texts[i]
    for j in keys:
        new_texts[i][j] = cur_texts[j].split("\n")
        new_texts[i][j] = [i for i in new_texts[i][j] if len(i)>0]

Определим метрику BLEU для пары текстов.

In [7]:
from nltk.translate import bleu_score
from nltk.translate.bleu_score import SmoothingFunction

In [8]:
def BLEU(text1, text2, smoothie = SmoothingFunction().method1):
    score = 0
    for ind, sent1 in enumerate(text1):
        sent2 = text2[ind]
        score += bleu_score.sentence_bleu(sent1, sent2, smoothing_function= smoothie)
    return score/len(text1)

In [9]:
import numpy as np
BLEU_table = {"_1":[], "_2":[]}
for i in BLEU_table:
    for j in keys:
        text1 = new_texts[i+".txt"][j]
        text2 = new_texts["_gold.txt"][j]
        BLEU_table[i].append(BLEU(text2, text1))
res = np.array([BLEU_table["_1"], BLEU_table["_2"]]).T

In [10]:
for ind, i in enumerate(keys):
    print(i, res[ind])

en_ru [0.01532306 0.0156166 ]
ru_kk [0.01161779 0.01114365]
en_kk [0.01297429 0.01225862]
ru_de [0.00545099 0.00539016]
de_en [0.00703511 0.00558257]


## 1. Лучший переводчик

Сейчас мы хотим выяснить, верно ли, что один из переводчиков лучше. Будем считать, что величины качества первого и второго переводчика есть случайные величины $T_1$ и $T_2$.

Проверим гипотезу $H_0: \mathbb{P}(T_1 > T_2) = \frac{1}{2}$ при альтернативе $H_1: \mathbb{P}(T_1 > T_2) \neq \frac{1}{2}$. Для этого воспользуемся критерием знаков при уровне значимости $\alpha = 0.5$.

Гипотеза отвергается, когда статистика этого критерия не лежит в диапазоне $[\frac{\alpha}{2}, 1-\frac{\alpha}{2}]$.

In [11]:
from scipy.stats import binom
def test(X1, X2, alpha = 0.05):
    T = 0
    for ind, i in enumerate(X1):
        T+= i >= X2[ind]
    p_value = (1-binom(len(X1), 0.5).cdf(T) )
    return p_value, alpha/2 <= p_value <= 1 - alpha/2

In [12]:
test(res[:, 0], res[:,1]) 

(0.03125, True)

Данные гипотезе противоречат. Значит, нельзя считать, что один из переводчиков лучше. 

## 2. Независимость

Проверим гипотезу о независимости при помощи коэффициента Кэндалла.

In [13]:
from scipy.stats import kendalltau
kendalltau(res[:, 0], res[:,1])

KendalltauResult(correlation=0.9999999999999999, pvalue=0.016666666666666666)

Коэффициент Кэндалла имеет достаточно высокое значение. Следовательно, качество перевода во многом обусловлено какая языковая пара обрабатывается. 